In [298]:
from pathlib import Path

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

## Constants

In [299]:
DATA_DIR = Path("../data")

SOLVERS = ["ortools", "cpoptimizer"]
TIME_LIMIT = [120, 900]

SET1 = ["JSP", "FJSP", "NW-PFSP", "NPFSP", "HFSP"]
SET2 = ["PFSP", "SDST-PFSP", "TCT-PFSP", "TT-PFSP"]
SET3 = ["RCPSP", "MMRCPSP", "RCMPSP"]

ALL = SET1 + SET2 + SET3

## Helpers

In [300]:
def present(df, index=ALL):
    """
    Put time limit and solver as MultiIndex and sort by time limit and OR-tools first.
    """
    unstacked = df.unstack(level=[2, 1])
    columns = sorted(unstacked.columns, key=lambda x: (x[0], -ord(x[1][0])))
    unstacked = unstacked[columns]
    return unstacked.loc[index]

In [301]:
def latex(df, float_format="%.2f"):
    text = df.to_latex(float_format=float_format)
    text = text.replace("time_limit", "Time Limit")
    text = text.replace("_", "-")
    text = text.replace("NaN", "-")
    print(text)

## Merging

In [302]:
bks = pd.read_csv(DATA_DIR / "bks.csv")
stats = pd.read_csv(DATA_DIR / "stats.csv")
results = pd.read_csv(DATA_DIR / "results.csv")

df = pd.merge(results, bks, on=["problem", "instance"])
# assert len(df) == len(results), "Not all results have a BKS entry."

df = pd.merge(df, stats, on=["problem", "instance"])
# assert len(df) == len(results), "Not all results have instance stats."

Debug cell to check which results don't have any bks. As long as these are "0.txt" instances, which are test instances, then it's fine.

In [303]:
merged = results.merge(bks, on=['problem', 'instance'], how='left', indicator=True)
not_in_bks = merged[merged['_merge'] == 'left_only']
not_in_bks['instance'].unique()

array(['0.txt'], dtype=object)

## Preprocessing

In [402]:
def calculate_rpd(row):
    if row["bks"] != 0:
        return ((row["objective"] - row["bks"]) / row["bks"]) * 100
    else:
        # When the BKS is zero and objective is zero, we set RPD to 0, 
        # otherwise set to 100.
        return 0 if row["objective"] == row["bks"] else 100

def calculate_gap(row):
    if row["objective"] != 0:
        return ((row["objective"] - row["lower_bound"]) / row["objective"]) * 100
    else:
        # When the objective is zero, we set RPD to 0, otherwise set to 100.
        return 0 if row["objective"] == row["bks"] else 100
        
def preprocess(df):
    df = df.copy()
    
    df["rpd"] = df.apply(calculate_rpd, axis=1)
    if "lower_bound" in df.columns:
        df["gap"] = df.apply(calculate_gap, axis=1)
    df["feas"] = df["status"].isin(["Optimal", "Feasible"])
    df["optimal"] = df["status"] == "Optimal"
    df["large"] = df["num_modes"] >= 5000
    
    return df

In [403]:
df = preprocess(df)

### Instance statistics

In [413]:
def agg_stats(x):
    return pd.Series({
        'min': x.min(),
        'avg': round(x.mean()),
        'max': x.max()
    })

# this ensures we only have each instance once (instead of 4x)
subset = df[(df['time_limit'] == 120) & (df['solver'] == 'ortools')] 

result = subset.groupby("problem").agg({
    'num_modes': ['count', 'min', 'mean', 'max'],
    # 'num_tasks': ['count', 'min', 'mean', 'max'],
    'num_resources': ['min', 'mean', 'max']
}).round(0)

columns = pd.MultiIndex.from_tuples([
    ('', 'instances'),
    ('Tasks', 'min'),
    ('Tasks', 'avg'),
    ('Tasks', 'max'),
    ('Resources', 'min'),
    ('Resources', 'avg'),
    ('Resources', 'max')
])

result.columns = columns
result = result.loc[ALL] # sort

for col in result.columns:
    result[col] = result[col].astype(int)

print(result.to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
 &  & \multicolumn{3}{r}{Tasks} & \multicolumn{3}{r}{Resources} \\
 & instances & min & avg & max & min & avg & max \\
problem &  &  &  &  &  &  &  \\
\midrule
JSP & 242 & 36 & 511 & 2000 & 5 & 15 & 20 \\
FJSP & 289 & 33 & 1581 & 14401 & 4 & 11 & 20 \\
NW-PFSP & 360 & 100 & 12610 & 48000 & 5 & 31 & 60 \\
NPFSP & 360 & 100 & 12610 & 48000 & 5 & 31 & 60 \\
HFSP & 1440 & 750 & 3750 & 10000 & 15 & 30 & 50 \\
PFSP & 120 & 100 & 1496 & 6000 & 5 & 19 & 60 \\
SDST-PFSP & 360 & 100 & 661 & 2000 & 5 & 12 & 20 \\
TCT-PFSP & 120 & 100 & 1496 & 6000 & 5 & 19 & 60 \\
TT-PFSP & 135 & 500 & 1500 & 2500 & 10 & 30 & 50 \\
RCPSP & 10080 & 32 & 122 & 302 & 3 & 4 & 4 \\
MMRCPSP & 3240 & 152 & 202 & 302 & 4 & 4 & 4 \\
RCMPSP & 2254 & 1488 & 1488 & 1488 & 4 & 4 & 4 \\
\bottomrule
\end{tabular}



### Feasibility

In [414]:
def percentage_feasible(feas):
    return sum(feas) / len(feas) * 100

groups = tl120.groupby(["problem", "solver"])
pct_feas = groups['feas'].apply(percentage_feasible)
latex(present(pct_feas), float_format= "%.2f")

NameError: name 'tl120' is not defined

### Optimal

In [406]:
def percentage_opt(row):
    return sum(row) / len(row) * 100

groups = df.groupby(["problem", "solver", "time_limit"])
pct_opt = groups['optimal'].apply(percentage_opt)
latex(present(pct_opt), float_format= "%.2f")

\begin{tabular}{lrrrr}
\toprule
Time Limit & \multicolumn{2}{r}{120} & \multicolumn{2}{r}{900} \\
solver & ortools & cpoptimizer & ortools & cpoptimizer \\
problem &  &  &  &  \\
\midrule
JSP & 33.06 & 36.36 & 41.32 & 47.11 \\
FJSP & 48.10 & 35.29 & 58.13 & 38.41 \\
NW-PFSP & 0.00 & 0.00 & 0.00 & 0.00 \\
NPFSP & 4.44 & 6.11 & 6.11 & 8.06 \\
HFSP & 0.00 & 0.00 & 0.14 & 0.00 \\
PFSP & 11.67 & 25.00 & 17.50 & 30.83 \\
SDST-PFSP & 0.00 & 0.00 & 0.00 & 1.11 \\
TCT-PFSP & 0.00 & 0.00 & 0.00 & 0.00 \\
TT-PFSP & 2.22 & 6.67 & 5.93 & 6.67 \\
RCPSP & 63.69 & 63.61 & 65.16 & 64.64 \\
MMRCPSP & 78.64 & 65.86 & 81.54 & 72.96 \\
RCMPSP & 9.14 & 8.70 & 9.94 & 8.78 \\
\bottomrule
\end{tabular}



### Relative percentage deviation

In [407]:
def rpd(df, groups=["problem", "solver", "time_limit"]):
    subset = df[df["feas"]] # drop rows with infeasible solutions
    grouped = subset.groupby(groups)
    return grouped['rpd'].mean().round(2)

In [408]:
res = present(rpd(df))
latex(res)
res

\begin{tabular}{lrrrr}
\toprule
Time Limit & \multicolumn{2}{r}{120} & \multicolumn{2}{r}{900} \\
solver & ortools & cpoptimizer & ortools & cpoptimizer \\
problem &  &  &  &  \\
\midrule
JSP & 3.83 & 2.75 & 1.98 & 1.80 \\
FJSP & 1.78 & 1.35 & 0.68 & 0.99 \\
NW-PFSP & 4.62 & 24.39 & 3.47 & 7.18 \\
NPFSP & 14.75 & 10.97 & 13.53 & 8.88 \\
HFSP & 24.66 & 8.63 & 13.34 & 6.58 \\
PFSP & 9.86 & 4.46 & 7.49 & 2.54 \\
SDST-PFSP & 13.96 & 5.48 & 8.82 & 4.41 \\
TCT-PFSP & 14.47 & 5.66 & 10.31 & 3.31 \\
TT-PFSP & 84.66 & 31.70 & 53.14 & 20.79 \\
RCPSP & 1.29 & 0.79 & 0.93 & 0.46 \\
MMRCPSP & 0.32 & 0.51 & 0.18 & 0.27 \\
RCMPSP & -0.42 & -0.70 & -0.52 & -0.85 \\
\bottomrule
\end{tabular}



time_limit     120                 900            
solver     ortools cpoptimizer ortools cpoptimizer
problem                                           
JSP           3.83        2.75    1.98        1.80
FJSP          1.78        1.35    0.68        0.99
NW-PFSP       4.62       24.39    3.47        7.18
NPFSP        14.75       10.97   13.53        8.88
HFSP         24.66        8.63   13.34        6.58
PFSP          9.86        4.46    7.49        2.54
SDST-PFSP    13.96        5.48    8.82        4.41
TCT-PFSP     14.47        5.66   10.31        3.31
TT-PFSP      84.66       31.70   53.14       20.79
RCPSP         1.29        0.79    0.93        0.46
MMRCPSP       0.32        0.51    0.18        0.27
RCMPSP       -0.42       -0.70   -0.52       -0.85

In [409]:
for instance_set in [SET1, SET2, SET3]:
    avg_gaps = res.loc[instance_set].mean().round(2).values
    text = '  &  '.join(str(gap) for gap in avg_gaps)
    print(f"& \\textit{{Average}} & {text} \\\\")

& \textit{Average} & 9.93  &  9.62  &  6.6  &  5.09 \\
& \textit{Average} & 30.74  &  11.82  &  19.94  &  7.76 \\
& \textit{Average} & 0.4  &  0.2  &  0.2  &  -0.04 \\


### RPD by instance size

In [411]:
tl900 = df[df['time_limit'] == 900]
res = rpd(tl900, ["problem", "solver", "large"])
res = present(res)
latex(res)
res

\begin{tabular}{lrrrr}
\toprule
large & \multicolumn{2}{r}{False} & \multicolumn{2}{r}{True} \\
solver & ortools & cpoptimizer & ortools & cpoptimizer \\
problem &  &  &  &  \\
\midrule
JSP & 1.98 & 1.80 & - & - \\
FJSP & 0.49 & 0.69 & 2.57 & 3.87 \\
NW-PFSP & 6.22 & 3.70 & 1.72 & 9.40 \\
NPFSP & 9.07 & 4.70 & 16.36 & 11.54 \\
HFSP & 8.63 & 5.42 & 23.72 & 9.12 \\
PFSP & 6.49 & 2.00 & 18.49 & 8.50 \\
SDST-PFSP & 8.82 & 4.41 & - & - \\
TCT-PFSP & 10.39 & 3.22 & 9.41 & 4.33 \\
TT-PFSP & 53.14 & 20.79 & - & - \\
RCPSP & 0.93 & 0.46 & - & - \\
MMRCPSP & 0.18 & 0.27 & - & - \\
RCMPSP & -0.52 & -0.85 & - & - \\
\bottomrule
\end{tabular}



large       False               True             
solver    ortools cpoptimizer ortools cpoptimizer
problem                                          
JSP          1.98        1.80     NaN         NaN
FJSP         0.49        0.69    2.57        3.87
NW-PFSP      6.22        3.70    1.72        9.40
NPFSP        9.07        4.70   16.36       11.54
HFSP         8.63        5.42   23.72        9.12
PFSP         6.49        2.00   18.49        8.50
SDST-PFSP    8.82        4.41     NaN         NaN
TCT-PFSP    10.39        3.22    9.41        4.33
TT-PFSP     53.14       20.79     NaN         NaN
RCPSP        0.93        0.46     NaN         NaN
MMRCPSP      0.18        0.27     NaN         NaN
RCMPSP      -0.52       -0.85     NaN         NaN

In [412]:
for instance_set in [SET1, SET2, SET3]:
    avg_gaps = res.loc[instance_set].mean().round(2).values
    text = '  &  '.join(str(gap) for gap in avg_gaps)
    print(f"& \\textit{{Average}} & {text} \\\\")

& \textit{Average} & 5.28  &  3.26  &  11.09  &  8.48 \\
& \textit{Average} & 19.71  &  7.6  &  13.95  &  6.42 \\
& \textit{Average} & 0.2  &  -0.04  &  nan  &  nan \\


## Optimality gap

In [314]:
def gap(df, groups=["problem", "solver", "time_limit"]):
    subset = df[df["feas"]] # drop rows with infeasible solutions
    grouped = subset.groupby(groups)
    return grouped['gap'].mean().round(2)

In [327]:
res = present(gap(df))
latex(res)
res

\begin{tabular}{lrrrr}
\toprule
Time Limit & \multicolumn{2}{r}{120} & \multicolumn{2}{r}{900} \\
solver & ortools & cpoptimizer & ortools & cpoptimizer \\
problem &  &  &  &  \\
\midrule
JSP & 5.27 & 5.11 & 3.40 & 4.09 \\
FJSP & 1.98 & 28.06 & 1.04 & 27.67 \\
NW-PFSP & 51.38 & 69.72 & 50.51 & 57.87 \\
NPFSP & 17.07 & 50.47 & 16.29 & 25.58 \\
HFSP & 19.47 & 67.20 & 11.94 & 66.54 \\
PFSP & 12.22 & 8.70 & 10.61 & 7.03 \\
SDST-PFSP & 34.59 & 28.94 & 30.75 & 28.24 \\
TCT-PFSP & 26.40 & 29.96 & 21.48 & 26.69 \\
TT-PFSP & 75.13 & 74.01 & 66.86 & 72.43 \\
RCPSP & 3.87 & 4.62 & 3.46 & 4.26 \\
MMRCPSP & 1.27 & 8.27 & 0.94 & 6.65 \\
RCMPSP & 33.80 & 67.43 & 14.91 & 67.36 \\
\bottomrule
\end{tabular}



time_limit     120                 900            
solver     ortools cpoptimizer ortools cpoptimizer
problem                                           
JSP           5.27        5.11    3.40        4.09
FJSP          1.98       28.06    1.04       27.67
NW-PFSP      51.38       69.72   50.51       57.87
NPFSP        17.07       50.47   16.29       25.58
HFSP         19.47       67.20   11.94       66.54
PFSP         12.22        8.70   10.61        7.03
SDST-PFSP    34.59       28.94   30.75       28.24
TCT-PFSP     26.40       29.96   21.48       26.69
TT-PFSP      75.13       74.01   66.86       72.43
RCPSP         3.87        4.62    3.46        4.26
MMRCPSP       1.27        8.27    0.94        6.65
RCMPSP       33.80       67.43   14.91       67.36

## Results

In [ ]:
columns = sorted(unstacked.columns, key=lambda x: (x[0], -ord(x[1][0])))
unstacked = unstacked[columns]
return unstacked.loc[index]

In [388]:
df = preprocess(df)
tl900 = df[(df["time_limit"] == 900) & df["feas"]]
res = tl900.groupby(["problem","solver"])[['rpd', 'gap']].mean()
unstacked = res.unstack()
columns = sorted(unstacked.columns, key=lambda x: (-ord(x[0][0]), -ord(x[1][0])))
unstacked = unstacked[columns]
res = unstacked.loc[ALL]
latex(res)

\begin{tabular}{lrrrr}
\toprule
 & \multicolumn{2}{r}{rpd} & \multicolumn{2}{r}{gap} \\
solver & ortools & cpoptimizer & ortools & cpoptimizer \\
problem &  &  &  &  \\
\midrule
JSP & 1.98 & 1.80 & 3.40 & 4.09 \\
FJSP & 0.68 & 0.99 & 1.04 & 27.67 \\
NW-PFSP & 3.47 & 7.18 & 50.51 & 57.87 \\
NPFSP & 13.53 & 8.88 & 16.29 & 25.58 \\
HFSP & 13.34 & 6.58 & 11.94 & 66.54 \\
PFSP & 7.49 & 2.54 & 10.61 & 7.03 \\
SDST-PFSP & 8.82 & 4.41 & 30.75 & 28.24 \\
TCT-PFSP & 10.31 & 3.31 & 21.48 & 26.69 \\
TT-PFSP & 53.14 & 20.79 & 66.86 & 72.43 \\
RCPSP & 0.93 & 0.46 & 3.46 & 4.26 \\
MMRCPSP & 0.18 & 0.27 & 0.94 & 6.65 \\
RCMPSP & -0.52 & -0.85 & 14.91 & 67.36 \\
\bottomrule
\end{tabular}



In [389]:
for instance_set in [SET1, SET2, SET3]:
    avg_gaps = res.loc[instance_set].mean().round(2).values
    text = '  &  '.join(str(gap) for gap in avg_gaps)
    print(f"& \\textit{{Average}} & {text} \\\\")

& \textit{Average} & 6.6  &  5.08  &  16.63  &  36.35 \\
& \textit{Average} & 19.94  &  7.76  &  32.43  &  33.6 \\
& \textit{Average} & 0.2  &  -0.04  &  6.44  &  26.09 \\


### Best-known solutions

In [316]:
idcs = df.groupby(["problem", "instance"])["rpd"].idxmin().values
best = df.loc[idcs]
improvements = best[best["rpd"] < 0].sort_values(["problem", "instance"])
improvements.groupby("problem").count()

,solver,time_limit,instance,status,objective,lower_bound,time,category,bks,num_modes,num_resources,num_tasks,rpd,gap,feas,optimal,large
problem,,,,,,,,,,,,,,,,,
FJSP,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30
MMRCPSP,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
NPFSP,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37
NW-PFSP,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40
RCMPSP,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025
RCPSP,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22
TT-PFSP,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [321]:
variants = ["MMRCPSP", "RCPSP"]
best = best[best["problem"].isin(variants)]
best[best["rpd"] < 0].sort_values(["problem", "instance"])

,problem,solver,time_limit,instance,status,objective,lower_bound,time,category,bks,num_modes,num_resources,num_tasks,rpd,gap,feas,optimal,large
57362,MMRCPSP,ortools,900,J10010_5.txt,Feasible,29.0,28.000000,900.02,MMLIB100,30.0,302,4,102,-3.333333,3.448276,True,False,False
57368,MMRCPSP,ortools,900,J10011_3.txt,Optimal,30.0,30.000000,655.08,MMLIB100,31.0,302,4,102,-3.225806,0.000000,True,True,False
57462,MMRCPSP,ortools,900,J1001_5.txt,Feasible,39.0,38.000000,900.06,MMLIB100,40.0,302,4,102,-2.500000,2.564103,True,False,False
57646,MMRCPSP,ortools,900,J10037_2.txt,Feasible,49.0,48.000000,900.24,MMLIB100,50.0,302,4,102,-2.000000,2.040816,True,False,False
57662,MMRCPSP,ortools,900,J10038_5.txt,Feasible,40.0,39.000000,900.03,MMLIB100,41.0,302,4,102,-2.439024,2.500000,True,False,False
54488,MMRCPSP,ortools,120,J10044_3.txt,Feasible,56.0,55.000000,120.06,MMLIB100,57.0,302,4,102,-1.754386,1.785714,True,False,False
57844,MMRCPSP,ortools,900,J10055_1.txt,Feasible,47.0,46.000000,900.08,MMLIB100,48.0,302,4,102,-2.083333,2.127660,True,False,False
57846,MMRCPSP,ortools,900,J10055_2.txt,Optimal,45.0,45.000000,483.77,MMLIB100,46.0,302,4,102,-2.173913,0.000000,True,True,False
58336,MMRCPSP,ortools,900,J1009_2.txt,Feasible,35.0,34.000000,900.02,MMLIB100,36.0,302,4,102,-2.777778,2.857143,True,False,False
58540,MMRCPSP,ortools,900,J5010_5.txt,Feasible,34.0,33.000000,900.02,MMLIB50,35.0,152,4,52,-2.857143,2.941176,True,False,False


## Extra

### Naderi FJSP

In [318]:
bks = pd.read_csv(DATA_DIR / "bks.csv")
stats = pd.read_csv(DATA_DIR / "stats.csv")
results = pd.read_csv(DATA_DIR / "fjsp_naderi.csv")

extra_df = pd.merge(results, bks, on=["problem", "instance"])
extra_df = pd.merge(extra_df, stats, on=["problem", "instance"])
extra_df = preprocess(extra_df)
extra_df["rpd"].mean()

7.553400719498263